# North Carolina Scorecard

This notebook generates the county, school district, and statewide "future voter scorecards" for NC. It is generalize to be updated every month, with minimal changes.

Scorecard outputs (tables) are written back to BigQuery, where they are then read into Google Sheets for formatting

In [6]:
import pandas as pd
import numpy as np
import pandas_gbq

## Inputs
Update the fields below each month

In [7]:
# Inputs
as_of_data_date = pd.Timestamp("2025-07-19")
acs_year = '2022' # 2022 for 2024 scorecards. ACS vintages trail by 2 years


## Outputs
Run the cells below, without edits each month

In [8]:
# Derived variables
latest_18_yob = as_of_data_date.year - 18 # 2006 for 2024 scorecards
earliest_18_yob = latest_18_yob - 1 # 2005 for 2024 scorecards

latest_45_yob = as_of_data_date.year - 45 # 1979 for 2024 scorecards
earliest_45_yob = latest_45_yob - 1 # 1978 for 2024 scorecards

data_date_suffix = str(as_of_data_date.year) + str(as_of_data_date.month).rjust(2, "0") + str(as_of_data_date.day).rjust(2, "0")  # this is the rolling "as of" date, where we snap the line for 18yos

# Define columns

    # Voter file
REG_YOB_LATE_YEAR = 'REG_YOB_' + str(latest_18_yob) # col name for number of registrants with given birth year (2006 for 2024 scorecards)
REG_YOB_EARLY_YEAR = 'REG_YOB_' + str(earliest_18_yob) # col name for number of registrants with given birth year (2005 for 2024 scorecards)

REG_45_PLUS_YOB_LATE_YEAR = 'REG_45_PLUS_YOB_' + str(latest_45_yob) # col name for number of registrants with given birth year (1979 for 2024 scorecards)
REG_45_PLUS_YOB_EARLY_YEAR = 'REG_45_PLUS_YOB_' + str(earliest_45_yob) # col name for number of registrants with given birth year (1978 for 2024 scorecards)

    # ACS
EST_18_YO_THIS_YEAR = 'EST_18_YO_' + str(as_of_data_date.year) # col name for number of 18 yos this "as_of" year estimated from ACS
EST_18_AND_19_YO_THIS_YEAR = 'EST_18_AND_19_YO_' + str(as_of_data_date.year) # col name for number of 18 and 19 yos this "as_of" year estimated from ACS

EST_45_PLUS_YO_THIS_YEAR = 'EST_45_PLUS_YO_' + str(as_of_data_date.year) # col name for number of 45+ yos in the "as of" year estimated from ACS


### County Scorecard

In [9]:
# Define table names
voter_file_table = data_date_suffix + "_scorecard_nc"
voter_source_table = data_date_suffix + "_nc_voter_registration_and_history"

acs_S0101_table = "S0101_us_counties_acs5y_" + acs_year

#### Query from BQ
This query:
* Summarizes the voter file by county, counting the number of registrants in a given birth year.
* Then, left joins the county estimates for the total number of 18 (and 19) yos from the ACS
    * The estimates for the total number of 18 (and 19) yos are derived from the raw estimates of 15-17 yos, **assuming a uniform distribution of population across 15, 16, and 17 year olds.**
    * Since the ACS trails by 2 years, the ACS estimate of 15-17yos is used as a proxy for the number of 17-19yos today. (This means we are intentionally *not* trying to count the college student or "group quarters" population in our denominator)

In [14]:

project_id = "tcc-research"

sql = f"""
WITH addresses AS (
  SELECT
    CAST(ncid AS STRING) AS VOTER_ID,
    CASE
      -- WATAUGA
      WHEN SAFE_CAST(county_id AS INT64) = 95 AND (mail_addr1 LIKE 'ASU %')
        THEN FARM_FINGERPRINT('APPALACHIAN STATE UNIVERSITY DORM')
      -- ORANGE
      WHEN SAFE_CAST(county_id AS INT64) = 68 AND res_city_desc = 'CHAPEL HILL'
        THEN FARM_FINGERPRINT(CONCAT(
              COALESCE(REGEXP_REPLACE(res_street_address, r'( #.*)$', ''), ''),
              COALESCE(res_city_desc, ''),
              COALESCE(state_cd, ''),
              COALESCE(zip_code, '')
            ))
      ELSE FARM_FINGERPRINT(CONCAT(
              COALESCE(res_street_address, ''),
              COALESCE(res_city_desc, ''),
              COALESCE(state_cd, ''),
              COALESCE(zip_code, '')
            ))
    END AS ADDRESS_ID
  FROM `tcc-research.nc_sources.{voter_source_table}`
),


young_voters AS (
  SELECT
    a.*,
    addresses.ADDRESS_ID
  FROM `tcc-research.nc_production.{voter_file_table}` a
  LEFT JOIN addresses
    ON CAST(addresses.VOTER_ID AS STRING) = CAST(a.VOTER_ID AS STRING)
  WHERE a.VOTER_STATUS IN ('ACTIVE')
),

address_count_18 AS (
  SELECT
    ADDRESS_ID,
    COUNT(VOTER_ID) AS N_18_VOTERS_AT_ADDRESS
  FROM young_voters
  WHERE YEAR_OF_BIRTH IN ({latest_18_yob}, {earliest_18_yob})
  GROUP BY ADDRESS_ID
),

voter_file_county AS (
  SELECT
    LPAD(CAST(COUNTY_FIPS AS STRING), 3, '0') AS COUNTY_FIPS_STR,
    COUNTY_NAME,
    COUNT(VOTER_ID) AS N_VOTERS,
    COUNTIF(YEAR_OF_BIRTH = {latest_18_yob})     AS {REG_YOB_LATE_YEAR},
    COUNTIF(YEAR_OF_BIRTH = {earliest_18_yob})   AS {REG_YOB_EARLY_YEAR},
    COUNTIF(YEAR_OF_BIRTH <= {latest_45_yob})    AS {REG_45_PLUS_YOB_LATE_YEAR},
    COUNTIF(YEAR_OF_BIRTH <= {earliest_45_yob})  AS {REG_45_PLUS_YOB_EARLY_YEAR}
  FROM young_voters
  LEFT JOIN address_count_18 USING (ADDRESS_ID)
  WHERE COALESCE(N_18_VOTERS_AT_ADDRESS, 0) < 4
  GROUP BY COUNTY_FIPS_STR, COUNTY_NAME
),

acs_county AS (
  SELECT
    -- ensure zero-padded string to match vfc
    LPAD(CAST(COUNTY_FIPS AS STRING), 3, '0') AS COUNTY_FIPS,
    EST_15_TO_17_YO,
    MOE_15_TO_17_YO,
    CAST(EST_15_TO_17_YO AS FLOAT64) / 3             AS {EST_18_YO_THIS_YEAR},
    CAST(EST_15_TO_17_YO AS FLOAT64) * 2 / 3         AS {EST_18_AND_19_YO_THIS_YEAR},
    (EST_45_TO_49_YO + EST_50_TO_54_YO + EST_55_TO_59_YO + EST_60_AND_OVER) AS {EST_45_PLUS_YO_THIS_YEAR}
  FROM `tcc-research.acs_sources.{acs_S0101_table}`
  -- if STATE_FIPS is STRING in your ACS table, compare to '37'
  WHERE CAST(STATE_FIPS AS INT64) = 37
)

SELECT
  vfc.COUNTY_FIPS_STR AS COUNTY_FIPS,
  vfc.COUNTY_NAME,
  vfc.N_VOTERS,
  vfc.{REG_YOB_LATE_YEAR},
  vfc.{REG_YOB_EARLY_YEAR},
  vfc.{REG_45_PLUS_YOB_LATE_YEAR},
  vfc.{REG_45_PLUS_YOB_EARLY_YEAR},
  acs.EST_15_TO_17_YO,
  acs.MOE_15_TO_17_YO,
  acs.{EST_18_YO_THIS_YEAR},
  acs.{EST_18_AND_19_YO_THIS_YEAR},
  acs.{EST_45_PLUS_YO_THIS_YEAR}
FROM voter_file_county vfc
LEFT JOIN acs_county acs
  ON vfc.COUNTY_FIPS_STR = acs.COUNTY_FIPS

"""
df = pandas_gbq.read_gbq(sql, project_id=project_id)


Downloading: 100%|██████████|


In [15]:
# Preview
df.head()

,COUNTY_FIPS,COUNTY_NAME,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025
0,081,GUILFORD,326427,2952,5374,190773,185903,21093,*****,7031.000000,14062.000000,219266
1,125,MOORE,69925,452,979,46312,45362,3658,121,1219.333333,2438.666667,48663
2,071,GASTON,135612,820,2091,82136,80147,9089,34,3029.666667,6059.333333,99717
3,001,ALAMANCE,101566,691,1734,61700,60285,6617,42,2205.666667,4411.333333,73778
4,059,DAVIE,29671,174,461,19632,19252,1693,86,564.333333,1128.666667,21560


In [16]:
df_reg_est = df.copy()

#### Metric 1: Estimated registration rate of 18 year olds as of a rolling date (i.e. latest month)
Ex: In March 2024, we consider the registration rate among those born between March 2nd 2005 and March 1st 2006


Notes:
- The MI voter file does *not* include full birth dates for registrants – only year of birth is included
- 18 yos as of a given date in the middle of the calendar year can have 2 potential years of birth. We refer to these as the "later 18 yo year of birth" (2006 for 2024 scorecards) and the "earlier 18 yo year of birth" (2005 for 2024 scorecards)
- MI voter file includes 18yo registrants only: Those who are 18 years old prior to March 1st 2024. This was confirmed by the SOS. This means we can assume everyone born in the "later 18 year old year" is 18 (and there are no 17 year old).
- We still need to discount those born in the "earlier 18-year old year", because some of those born in that year are already 19
    - Ex: in March 2024, those born in Jan (31 days) and Feb (28 days) 2005 are already 19, so only those born March through December 2005 are 18

Estimation:

To estimate the number of 18 yos as of a rolling date, we "pro-rate" the number of registrants born in a given year based on the share of days in the year that could be 18yo birthdays. There are two steps:
- For the later 18 yo year of birth: Assume all are 18
- For the earlier 18 yo year of birth: Estimate the number of days that could be 18yo birthdays. Calculate the number of total potential birthdays included in the voter file (just ~365). Calculate the ratio of these numbers.


Assumptions:
- Voter file is as of 1st of the month (confirmed by SOS)
- Even distribution of birthdays across all days of year
- Uniform registration rates among older 18 yos, and younger 19yos

In [17]:
# Define column names
EST_REG_18_YO_AS_OF_ROLLING = 'EST_REG_18_YO_AS_OF_' + data_date_suffix # col name for estimated 18yo as of rolling date

# Birthday splits 18 yo vs. 19 yo in voter file (hypothetical)
earliest_bday_18 = as_of_data_date - pd.tseries.offsets.DateOffset(years=19) + pd.tseries.offsets.DateOffset(days=1) # earliest possible bday for 18yo

n_bdays_of_18_early = pd.Timestamp(str(earliest_18_yob) +"-12-31") - earliest_bday_18 # number of possible bdays of 18 yos in earlier 18 yo year of birth
n_total_days_of_early_year = pd.Timestamp(str(earliest_18_yob) +"-12-31") - pd.Timestamp(str(earliest_18_yob) +"-01-01") # number of total birthdays in earlier 18 yo year of birth (should be 365)

# Discounts
    # Share of 18yo in late year
share_18_late_year = 1

    # Share of 18yo in early year
share_18_early_year = n_bdays_of_18_early / n_total_days_of_early_year

    # CHECKS
print("share of 18 yo in late year: {}".format(share_18_late_year))
print("share of 18 yo in early year: {}".format(share_18_early_year))

share of 18 yo in late year: 1
share of 18 yo in early year: 0.45054945054945056


In [18]:
# Calculate numerator (registrants)
df_reg_est[EST_REG_18_YO_AS_OF_ROLLING] = df_reg_est[REG_YOB_LATE_YEAR] * share_18_late_year  + df_reg_est[REG_YOB_EARLY_YEAR] * share_18_early_year

# Calculate estimated registration rate
EST_REG_RATE_18_YO_AS_OF_ROLLING = 'EST_REG_RATE_18_YO_AS_OF_' + data_date_suffix # col name for estimated 18yo as of rolling date
df_reg_est[EST_REG_RATE_18_YO_AS_OF_ROLLING] = df_reg_est[EST_REG_18_YO_AS_OF_ROLLING] / df_reg_est[EST_18_YO_THIS_YEAR] # estimated registered 18yo over ACS 18yo population estimate

In [19]:
df_reg_est = df_reg_est.sort_values('N_VOTERS', ascending=False)
df_reg_est.head()

,COUNTY_FIPS,COUNTY_NAME,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025,EST_REG_18_YO_AS_OF_20250719,EST_REG_RATE_18_YO_AS_OF_20250719
22,183,WAKE,736513,5350,12678,401320,388463,47558,30,15852.666667,31705.333333,429851,11062.065934,0.697805
86,119,MECKLENBURG,684230,6723,10642,348288,337131,42911,18,14303.666667,28607.333333,401215,11517.747253,0.80523
0,081,GUILFORD,326427,2952,5374,190773,185903,21093,*****,7031.000000,14062.000000,219266,5373.252747,0.764223
68,067,FORSYTH,228851,1648,3634,136317,133085,15559,27,5186.333333,10372.666667,161271,3285.296703,0.633453
67,063,DURHAM,197060,1072,2535,100978,97992,10224,39,3408.000000,6816.000000,120957,2214.142857,0.64969


#### Metric 2: Estimated registration rate of 45 year olds as of a rolling date (i.e. latest month)
To count the 45+ yo as of a rolling date, we need to discount some folks born in the latest year of 45 year olds, because they are still 44

Assumptions:
- Even distribution of birthdays across all days of year
- Uniform registration rates among older 44 yos, and younger 45yos

In [20]:
# Define column names
EST_REG_45_PLUS_YO_AS_OF_ROLLING = 'EST_REG_45_PLUS_YO_AS_OF_' + data_date_suffix # col name for estimated 45yo as of rolling date

# Birthday splits 44 yo vs. 45 yo in voter file (hypothetical)
lastest_bday_45 = as_of_data_date - pd.tseries.offsets.DateOffset(years=45)  # latest possible bday for 45yo

n_bdays_of_45 = lastest_bday_45 - pd.Timestamp(str(latest_45_yob) +"-01-01") # number of possible bdays of 45 yos in later 45yo year of birth
n_total_days_of_late_year = pd.Timestamp(str(latest_45_yob) +"-12-31") - pd.Timestamp(str(latest_45_yob) +"-01-01") # number of total birthdays in later 45yo year of birth


    # Share of 45yo in early year
share_45_late_year = n_bdays_of_45 / n_total_days_of_late_year

    # CHECKS
print("share of 45 yo in early year: {}".format(share_45_late_year))

share of 45 yo in early year: 0.547945205479452


In [21]:
# Calculate numerator (registrants)
df_reg_est[EST_REG_45_PLUS_YO_AS_OF_ROLLING] = ((df_reg_est[REG_45_PLUS_YOB_LATE_YEAR] - df_reg_est[REG_45_PLUS_YOB_EARLY_YEAR]) * share_45_late_year)  + df_reg_est[REG_45_PLUS_YOB_EARLY_YEAR]

# Calculate estimated registration rate
EST_REG_RATE_45_PLUS_YO_AS_OF_ROLLING = 'EST_REG_RATE_45_PLUS_YO_AS_OF_' + data_date_suffix # col name for estimated 45yo as of rolling date
df_reg_est[EST_REG_RATE_45_PLUS_YO_AS_OF_ROLLING] = df_reg_est[EST_REG_45_PLUS_YO_AS_OF_ROLLING] / df_reg_est[EST_45_PLUS_YO_THIS_YEAR] # estimated registered 45yo over ACS 45yo population estimate

In [22]:
df_reg_est = df_reg_est.sort_values('N_VOTERS', ascending=False)
df_reg_est.head(20)

,COUNTY_FIPS,COUNTY_NAME,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025,EST_REG_18_YO_AS_OF_20250719,EST_REG_RATE_18_YO_AS_OF_20250719,EST_REG_45_PLUS_YO_AS_OF_20250719,EST_REG_RATE_45_PLUS_YO_AS_OF_20250719
22,183,WAKE,736513,5350,12678,401320,388463,47558,30,15852.666667,31705.333333,429851,11062.065934,0.697805,395507.931507,0.920105
86,119,MECKLENBURG,684230,6723,10642,348288,337131,42911,18,14303.666667,28607.333333,401215,11517.747253,0.80523,343244.424658,0.855512
0,081,GUILFORD,326427,2952,5374,190773,185903,21093,*****,7031.000000,14062.000000,219266,5373.252747,0.764223,188571.493151,0.860012
68,067,FORSYTH,228851,1648,3634,136317,133085,15559,27,5186.333333,10372.666667,161271,3285.296703,0.633453,134855.958904,0.836207
67,063,DURHAM,197060,1072,2535,100978,97992,10224,39,3408.000000,6816.000000,120957,2214.142857,0.64969,99628.164384,0.823666
54,021,BUNCOMBE,178679,952,2100,110111,107209,8663,38,2887.666667,5775.333333,126134,1898.153846,0.657331,108799.136986,0.862568
44,051,CUMBERLAND,172245,1245,3006,97425,94754,12860,36,4286.666667,8573.333333,111430,2599.351648,0.606381,96217.561644,0.86348
13,179,UNION,156545,1532,3387,94428,91518,12970,5,4323.333333,8646.666667,99429,3058.010989,0.707327,93112.520548,0.936472
39,129,NEW HANOVER,155945,784,2125,93842,91606,7279,81,2426.333333,4852.666667,100255,1741.417582,0.717716,92831.205479,0.925951
48,101,JOHNSTON,143392,1070,2627,82796,80349,9984,98,3328.000000,6656.000000,88941,2253.593407,0.677161,81689.821918,0.918472


In [24]:
df_reg_est = df_reg_est.sort_values('EST_REG_RATE_18_YO_AS_OF_20250719', ascending=False)
df_reg_est

,COUNTY_FIPS,COUNTY_NAME,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025,EST_REG_18_YO_AS_OF_20250719,EST_REG_RATE_18_YO_AS_OF_20250719,EST_REG_45_PLUS_YO_AS_OF_20250719,EST_REG_RATE_45_PLUS_YO_AS_OF_20250719
70,189,WATAUGA,34007,183,416,17707,17283,1202,29,400.666667,801.333333,20528,370.428571,0.924531,17515.328767,0.853241
90,003,ALEXANDER,23177,256,345,15084,14790,1341,121,447.000000,894.000000,17721,411.43956,0.920446,14951.09589,0.843694
8,177,TYRRELL,1881,1,21,1361,1334,36,39,12.000000,24.000000,1709,10.461538,0.871795,1348.794521,0.78923
89,149,POLK,14678,75,167,10805,10658,520,30,173.333333,346.666667,11771,150.241758,0.866779,10738.547945,0.912289
93,141,PENDER,44743,358,706,28364,27577,2379,130,793.000000,1586.000000,28767,676.087912,0.85257,28008.232877,0.973624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,185,WARREN,11445,45,144,8271,8110,681,70,227.000000,454.000000,9962,109.879121,0.484049,8198.219178,0.822949
71,033,CASWELL,13455,50,156,9238,9077,783,46,261.000000,522.000000,11671,120.285714,0.460865,9165.219178,0.785299
32,173,SWAIN,8221,38,110,5483,5345,584,98,194.666667,389.333333,6329,87.56044,0.449797,5420.616438,0.856473
20,103,JONES,6144,22,78,4193,4093,392,71,130.666667,261.333333,4952,57.142857,0.437318,4147.794521,0.8376


#### Output
Write back to BQ

##### Wide

In [25]:
# Flag largest counties
df_reg_est['is_in_10_largest'] = np.where(df_reg_est.COUNTY_NAME.isin(df_reg_est.nlargest(10, columns='EST_18_YO_2025').COUNTY_NAME),1,0)

In [26]:
# write
project_id = "tcc-research"
table_id = 'nc_output.' + data_date_suffix+ '_nc_county_scorecard_output'

pandas_gbq.to_gbq(df_reg_est, table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<?, ?it/s]


### Statewide Scorecard

In [27]:
# Define table names
voter_file_table = data_date_suffix + "_scorecard_nc"
acs_S0101_table = "S0101_us_states_acs5y_" + acs_year

#### Query from BQ
This query:
* Summarizes the voter file for NC state, counting the number of registrants in a given birth year.
* Then, left joins the statewide estimates for the total number of 18 (and 19) yos from the ACS
    * The estimates for the total number of 18 (and 19) yos is derived from the raw estimates of 15-17 yos, **assuming a uniform distribution of population across 15, 16, and 17 year olds.**
    * Since the ACS trails by 2 years, the ACS estimate of 15-17yos is used as a proxy for the number of 17-19yos today. (This means we are intentionally *not* trying to count the college student or "group quarters" population in our denominator)

In [28]:
# Define GCP project
project_id = "tcc-research"

# Define query, including variables and column names that adjust with time
sql = """
WITH addresses AS(
    SELECT
    ncid as VOTER_ID,
    CASE 
        -- WATAUGA 
        WHEN county_id = '95' AND (mail_addr1 LIKE 'ASU %') THEN FARM_FINGERPRINT('APPALACHIAN STATE UNIVERSITY DORM')
        -- ORANGE 
        WHEN county_id = '68' AND res_city_desc = 'CHAPEL HILL' THEN FARM_FINGERPRINT(CONCAT(COALESCE(REGEXP_REPLACE(res_street_address, '( #.*)$', '') ,''), COALESCE(res_city_desc,''), COALESCE(state_cd,''), COALESCE(zip_code,'')))
   
        ELSE FARM_FINGERPRINT(CONCAT(COALESCE(res_street_address,''), COALESCE(res_city_desc,''), COALESCE(state_cd,''), COALESCE(zip_code,'')))
    END as ADDRESS_ID
    FROM `tcc-research.nc_sources.""" + voter_source_table + """`


), young_voters AS(
    
    SELECT  
    a.*,
    ADDRESS_ID 
    FROM `tcc-research.nc_production.""" + voter_file_table + """` a
    LEFT JOIN addresses on addresses.VOTER_ID = a.VOTER_ID
    WHERE VOTER_STATUS IN ('ACTIVE')

),address_count_18 AS(    
    SELECT  
    ADDRESS_ID,
    COUNT(VOTER_ID) AS N_18_VOTERS_AT_ADDRESS
    FROM young_voters a
    WHERE YEAR_OF_BIRTH IN (""" + str(latest_18_yob) + ", " + str(earliest_18_yob) + """)
    GROUP BY ADDRESS_ID

),voter_file_nc AS(
    SELECT
    STATE_FIPS,
    COUNT(VOTER_ID) AS N_VOTERS,
    COUNTIF(YEAR_OF_BIRTH = """ + str(latest_18_yob) + ") AS " + REG_YOB_LATE_YEAR + """,
    COUNTIF(YEAR_OF_BIRTH = """ + str(earliest_18_yob) + ") AS " + REG_YOB_EARLY_YEAR + """,
    COUNTIF(YEAR_OF_BIRTH <= """ + str(latest_45_yob) + ") AS " + REG_45_PLUS_YOB_LATE_YEAR + """,
    COUNTIF(YEAR_OF_BIRTH <= """ + str(earliest_45_yob) + ") AS " + REG_45_PLUS_YOB_EARLY_YEAR + """,
    FROM young_voters
    LEFT JOIN address_count_18 on young_voters.ADDRESS_ID = address_count_18.ADDRESS_ID
    WHERE COALESCE(N_18_VOTERS_AT_ADDRESS, 0)<4
    GROUP BY STATE_FIPS

), acs_nc AS(
    SELECT
    STATE_FIPS,
    EST_15_TO_17_YO,
    MOE_15_TO_17_YO,
    EST_15_TO_17_YO / 3 AS """ + EST_18_YO_THIS_YEAR + """,
    EST_15_TO_17_YO * 2 / 3 AS """ + EST_18_AND_19_YO_THIS_YEAR + """,
    EST_45_TO_49_YO + EST_50_TO_54_YO + EST_55_TO_59_YO + EST_55_TO_59_YO + EST_60_AND_OVER AS """ + EST_45_PLUS_YO_THIS_YEAR + """
    FROM `tcc-research.acs_sources.""" + acs_S0101_table + """`
    WHERE STATE_FIPS = "37"

)

SELECT
voter_file_nc.*,
acs_nc.EST_15_TO_17_YO,
acs_nc.MOE_15_TO_17_YO,
acs_nc.""" + EST_18_YO_THIS_YEAR + """,
acs_nc.""" + EST_18_AND_19_YO_THIS_YEAR + """,
acs_nc.""" + EST_45_PLUS_YO_THIS_YEAR + """

FROM voter_file_nc LEFT JOIN acs_nc ON voter_file_nc.STATE_FIPS = acs_nc.STATE_FIPS
"""
# Query
df = pandas_gbq.read_gbq(sql, project_id=project_id)

Downloading: 100%|██████████|


In [29]:
# Preview
df

,STATE_FIPS,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025
0,37,6505541,45502,103389,3908114,3811457,404849,794,134949.666667,269899.333333,5139416


In [30]:
df_reg_est = df.copy()

#### Metric 1: Estimated registration rate of 18 year olds as of a rolling date (i.e. latest month)
Ex: In March 2024, we consider the registration rate among those born between March 2nd 2005 and March 1st 2006


Notes:
- The MI voter file does *not* include full birth dates for registrants – only year of birth is included
- 18 yos as of a given date in the middle of the calendar year can have 2 potential years of birth. We refer to these as the "later 18 yo year of birth" (2006 for 2024 scorecards) and the "earlier 18 yo year of birth" (2005 for 2024 scorecards)
- MI voter file includes 18yo registrants only: Those who are 18 years old prior to March 1st 2024. This was confirmed by the SOS. This means we can assume everyone born in the "later 18 year old year" is 18 (and there are no 17 year old).
- We still need to discount those born in the "earlier 18-year old year", because some of those born in that year are already 19
    - Ex: in March 2024, those born in Jan (31 days) and Feb (28 days) 2005 are already 19, so only those born March through December 2005 are 18

Estimation:

To estimate the number of 18 yos as of a rolling date, we "pro-rate" the number of registrants born in a given year based on the share of days in the year that could be 18yo birthdays. There are two steps:
- For the later 18 yo year of birth: Assume all are 18
- For the earlier 18 yo year of birth: Estimate the number of days that could be 18yo birthdays. Calculate the number of total potential birthdays included in the voter file (just ~365). Calculate the ratio of these numbers.


Assumptions:
- Voter file is as of 1st of the month (confirmed by SOS)
- Even distribution of birthdays across all days of year
- Uniform registration rates among older 18 yos, and younger 19yos

In [31]:
# Define column names
EST_REG_18_YO_AS_OF_ROLLING = 'EST_REG_18_YO_AS_OF_' + data_date_suffix # col name for estimated 18yo as of rolling date

# Birthday splits 18 yo vs. 19 yo in voter file (hypothetical)
earliest_bday_18 = as_of_data_date - pd.tseries.offsets.DateOffset(years=19) + pd.tseries.offsets.DateOffset(days=1) # earliest possible bday for 18yo

n_bdays_of_18_early = pd.Timestamp(str(earliest_18_yob) +"-12-31") - earliest_bday_18 # number of possible bdays of 18 yos in earlier 18 yo year of birth
n_total_days_of_early_year = pd.Timestamp(str(earliest_18_yob) +"-12-31") - pd.Timestamp(str(earliest_18_yob) +"-01-01") # number of total birthdays in earlier 18 yo year of birth (should be 365)

# Discounts
    # Share of 18yo in late year
share_18_late_year = 1

    # Share of 18yo in early year
share_18_early_year = n_bdays_of_18_early / n_total_days_of_early_year

    # CHECKS
print("share of 18 yo in late year: {}".format(share_18_late_year))
print("share of 18 yo in early year: {}".format(share_18_early_year))

share of 18 yo in late year: 1
share of 18 yo in early year: 0.45054945054945056


In [32]:
# Calculate numerator (registrants)
df_reg_est[EST_REG_18_YO_AS_OF_ROLLING] = df_reg_est[REG_YOB_LATE_YEAR] * share_18_late_year  + df_reg_est[REG_YOB_EARLY_YEAR] * share_18_early_year

# Calculate estimated registration rate
EST_REG_RATE_18_YO_AS_OF_ROLLING = 'EST_REG_RATE_18_YO_AS_OF_' + data_date_suffix # col name for estimated 18yo as of rolling date
df_reg_est[EST_REG_RATE_18_YO_AS_OF_ROLLING] = df_reg_est[EST_REG_18_YO_AS_OF_ROLLING] / df_reg_est[EST_18_YO_THIS_YEAR] # estimated registered 18yo over ACS 18yo population estimate

In [33]:
df_reg_est = df_reg_est.sort_values('N_VOTERS', ascending=False)
df_reg_est.head()

,STATE_FIPS,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025,EST_REG_18_YO_AS_OF_20250719,EST_REG_RATE_18_YO_AS_OF_20250719
0,37,6505541,45502,103389,3908114,3811457,404849,794,134949.666667,269899.333333,5139416,92083.857143,0.682357


#### Metric 2: Estimated registration rate of 45 year olds as of a rolling date (i.e. latest month)
To count the 45+ yo as of a rolling date, we need to discount some folks born in the latest year of 45 year olds, because they are still 44

Assumptions:
- Even distribution of birthdays across all days of year
- Uniform registration rates among older 44 yos, and younger 45yos

In [34]:
# Define column names
EST_REG_45_PLUS_YO_AS_OF_ROLLING = 'EST_REG_45_PLUS_YO_AS_OF_' + data_date_suffix # col name for estimated 45yo as of rolling date

# Birthday splits 44 yo vs. 45 yo in voter file (hypothetical)
lastest_bday_45 = as_of_data_date - pd.tseries.offsets.DateOffset(years=45)  # latest possible bday for 45yo

n_bdays_of_45 = lastest_bday_45 - pd.Timestamp(str(latest_45_yob) +"-01-01") # number of possible bdays of 45 yos in later 45yo year of birth
n_total_days_of_late_year = pd.Timestamp(str(latest_45_yob) +"-12-31") - pd.Timestamp(str(latest_45_yob) +"-01-01") # number of total birthdays in later 45yo year of birth


    # Share of 45yo in early year
share_45_late_year = n_bdays_of_45 / n_total_days_of_late_year

    # CHECKS
print("share of 45 yo in early year: {}".format(share_45_late_year))

share of 45 yo in early year: 0.547945205479452


In [35]:
# Calculate numerator (registrants)
df_reg_est[EST_REG_45_PLUS_YO_AS_OF_ROLLING] = ((df_reg_est[REG_45_PLUS_YOB_LATE_YEAR] - df_reg_est[REG_45_PLUS_YOB_EARLY_YEAR]) * share_45_late_year)  + df_reg_est[REG_45_PLUS_YOB_EARLY_YEAR]

# Calculate estimated registration rate
EST_REG_RATE_45_PLUS_YO_AS_OF_ROLLING = 'EST_REG_RATE_45_PLUS_YO_AS_OF_' + data_date_suffix # col name for estimated 45yo as of rolling date
df_reg_est[EST_REG_RATE_45_PLUS_YO_AS_OF_ROLLING] = df_reg_est[EST_REG_45_PLUS_YO_AS_OF_ROLLING] / df_reg_est[EST_45_PLUS_YO_THIS_YEAR] # estimated registered 45yo over ACS 45yo population estimate

In [36]:
df_reg_est = df_reg_est.sort_values('N_VOTERS', ascending=False)
df_reg_est.head()

,STATE_FIPS,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025,EST_REG_18_YO_AS_OF_20250719,EST_REG_RATE_18_YO_AS_OF_20250719,EST_REG_45_PLUS_YO_AS_OF_20250719,EST_REG_RATE_45_PLUS_YO_AS_OF_20250719
0,37,6505541,45502,103389,3908114,3811457,404849,794,134949.666667,269899.333333,5139416,92083.857143,0.682357,3864419.739726,0.751918


#### Output
Write back to BQ

In [37]:
# write
project_id = "tcc-research"
table_id = 'nc_output.' + data_date_suffix+ '_nc_statewide_scorecard_output'

pandas_gbq.to_gbq(df_reg_est, table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 947.87it/s]
